In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [3]:
filenames = os.listdir('/kaggle/input/dogs-vs-cats/train/train/')

In [4]:
classes = []
for filename in filenames:
    classes.append(filename[:3])
classes[:5]

['cat', 'dog', 'dog', 'dog', 'dog']

In [5]:
df = pd.DataFrame({"FILENAME":filenames,'CLASS':classes})

In [6]:
df.head()

,FILENAME,CLASS
0,cat.12461.jpg,cat
1,dog.3443.jpg,dog
2,dog.7971.jpg,dog
3,dog.10728.jpg,dog
4,dog.1942.jpg,dog


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
data_gen = ImageDataGenerator(zoom_range=0.2,
                              rotation_range=0.2,
                              horizontal_flip=True,
                              rescale=1/255)

In [9]:
train_data = data_gen.flow_from_dataframe(df,'/kaggle/input/dogs-vs-cats/train/train/',
                             x_col='FILENAME',
                             y_col='CLASS',
                             target_size=(224,224),batch_size=64)

Found 25000 validated image filenames belonging to 2 classes.


In [10]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from tensorflow.keras.applications import VGG16

In [11]:
vgg16 = VGG16(include_top=False,input_shape=(224,224,3))

2023-01-06 16:53:38.619547: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 16:53:38.620491: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 16:53:38.806807: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 16:53:38.807684: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 16:53:38.808468: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

58900480/58889256 [==============================] - 0s 0us/step


In [12]:
# TRaining off for existing layers of VGG16
for layer in vgg16.layers:
    layer.trainable = False

In [13]:
last_layer = Dense(2,activation='softmax')(Flatten()(vgg16.output))

In [14]:
model = Model(inputs = vgg16.input, outputs = last_layer)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [16]:
# model = Sequential()
# model.add(Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)))
# model.add(MaxPool2D())
# model.add(Conv2D(64,(3,3),activation='relu'))
# model.add(MaxPool2D())
# model.add(Conv2D(128,(3,3),activation='relu'))
# model.add(MaxPool2D())
# model.add(Conv2D(128,(5,5),activation='relu'))
# model.add(MaxPool2D())
# model.add(Conv2D(256,(3,3),activation='relu'))
# model.add(MaxPool2D())
# model.add(Flatten())
# model.add(Dense(2,activation='softmax'))



In [17]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [18]:
model.fit(train_data, epochs=5)

2023-01-06 16:53:44.395774: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5


2023-01-06 16:53:47.714662: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


391/391 [==============================] - 501s 1s/step - loss: 0.6129 - accuracy: 0.8215
Epoch 2/5
391/391 [==============================] - 383s 978ms/step - loss: 0.4074 - accuracy: 0.8822
Epoch 3/5
391/391 [==============================] - 382s 976ms/step - loss: 0.3688 - accuracy: 0.8976
Epoch 4/5
391/391 [==============================] - 378s 965ms/step - loss: 0.3547 - accuracy: 0.9018
Epoch 5/5
391/391 [==============================] - 378s 965ms/step - loss: 0.3230 - accuracy: 0.9105


In [19]:
train_data.class_indices

{'cat': 0, 'dog': 1}

In [21]:
model.save('cnn_own_vgg16.h5')